In [1]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import split



In [3]:
host, port = ("127.0.0.1", "65432")

spark = SparkSession\
    .builder\
    .appName("DiabetesTest")\
    .getOrCreate()

lines = spark\
    .readStream\
    .format('socket')\
    .option('host', host)\
    .option('port', port)\
    .load()

CSVData = lines.select(\
        split(lines.value, ',').getItem(0).alias('eventTime').cast("timestamp"),\
        split(lines.value, ',').getItem(1).alias('bloodGlucose').cast("int"),\
        split(lines.value, ',').getItem(2).alias('deviceID'),\
        )

selectAndFilter = CSVData.select("eventTime","bloodGlucose")\
        .where("bloodGlucose > 0")
    
query = selectAndFilter\
    .writeStream\
    .outputMode('append')\
    .format('console')\
    .start()

24/04/14 19:07:13 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/04/14 19:07:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-880b62bb-d105-4c4d-aecd-4c0f8b32694b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/14 19:07:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+------------+
|eventTime|bloodGlucose|
+---------+------------+
+---------+------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+------------+
|           eventTime|bloodGlucose|
+--------------------+------------+
|2024-04-14 19:07:...|         120|
+--------------------+------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+------------+
|           eventTime|bloodGlucose|
+--------------------+------------+
|2024-04-14 19:07:...|         120|
+--------------------+------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+------------+
|           eventTime|bloodGlucose|
+--------------------+------------+
|2024-04-14 19:07:...|         120|
+----

24/04/14 19:07:24 WARN TextSocketMicroBatchStream: Stream closed by 127.0.0.1:65432


In [4]:
query.stop()